In [2]:
import tkinter as tk
import cv2
import mediapipe as mp
from PIL import Image, ImageTk
import pyautogui
from pynput.keyboard import Key, Controller
import subprocess

class CameraApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Hand Detection")
        self.root.geometry("1280x720")  # Set window size

        self.main_frame = tk.Frame(root)
        self.main_frame.pack(expand=True, fill=tk.BOTH)

        self.start_button = tk.Button(self.main_frame, text="Start", command=self.open_camera)
        self.start_button.pack(pady=10)

        self.stop_button = tk.Button(self.main_frame, text="Stop", command=self.stop_camera)
        self.stop_button.pack(pady=10)

        self.cap = None  # Initialize camera capture object
        self.frame_width = None
        self.frame_height = None

        self.hand_detector = mp.solutions.hands.Hands()
        self.drawing_utils = mp.solutions.drawing_utils

        self.status_label = tk.Label(self.main_frame, text="Hand Status: ")
        self.status_label.pack(pady=5)
        self.hand_status = tk.StringVar()
        self.hand_status.set("Unknown")
        self.status_display = tk.Label(self.main_frame, textvariable=self.hand_status)
        self.status_display.pack(pady=5)

        self.camera_label = tk.Label(self.main_frame)
        self.camera_label.pack()

    def open_camera(self):
        if self.cap is None or not self.cap.isOpened():
            self.cap = cv2.VideoCapture(0)
            self.show_frame()
        else:
            print("Camera is already open.")

    def show_frame(self):
        ret, frame = self.cap.read()
        if ret:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = cv2.flip(frame, 1)  # Flip frame horizontally for mirror effect
            frame = cv2.resize(frame, (960, 540))  # Resize frame
            self.frame_height, self.frame_width, _ = frame.shape
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Detect hands
            output = self.hand_detector.process(rgb_frame)
            hand_status = self.detect_hand_status(output)
            self.hand_status.set(hand_status)

            if hand_status == "Closed":
                self.increase_volume()
            elif hand_status == "Open":
                self.decrease_volume()  # Decrease volume when hand is open

            # if hand_status == "Thumb Up":  # Perform play/pause when thumb is up
            #     self.play_previous()
            
            if output.multi_hand_landmarks:
                for hand_landmarks in output.multi_hand_landmarks:
                    self.drawing_utils.draw_landmarks(frame, hand_landmarks, mp.solutions.hands.HAND_CONNECTIONS)

            # Convert the frame to ImageTk format and display it
            img = Image.fromarray(frame)
            img = ImageTk.PhotoImage(img)
            self.camera_label.configure(image=img)
            self.camera_label.image = img

            # Repeat the process
            self.camera_label.after(10, self.show_frame)
        else:
            print("Failed to capture frame.")

    def detect_hand_status(self, output):
        if output.multi_hand_landmarks:
            for hand_landmarks in output.multi_hand_landmarks:
                landmarks = hand_landmarks.landmark
                thumb_tip = landmarks[4]
                index_tip = landmarks[8]
                thumb_y = thumb_tip.y * self.frame_height
                index_y = index_tip.y * self.frame_height

                # Calculate distance between thumb and index finger
                distance = abs(thumb_y - index_y)

                if distance < 50:
                    return "Closed"
                else:
                    return "Open"
        return "Unknown"
        
    def increase_volume(self):
            subprocess.run(["osascript", "-e", "set volume output volume (output volume of (get volume settings) + 5)"])
    def decrease_volume(self):
            subprocess.run(["osascript", "-e", "set volume output volume (output volume of (get volume settings) - 5)"])
    # def play_previous():
    #     subprocess.run(["osascript", "-e", "tell application \"System Events\" to key code 97 using {fn down}"])

    def stop_camera(self):
        if self.cap is not None and self.cap.isOpened():
            self.cap.release()
        else:
            print("Camera is not open.")

root = tk.Tk()
app = CameraApp(root)
root.mainloop()

I0000 00:00:1709709742.938422 6501580 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1 Pro


Failed to capture frame.
